# Bank Marketing Dataset

Ultima campaña de marketing que realizó el banco e identificar los patrones que nos ayudarán a extraer conclusiones para desarrollar estrategias futuras.

**Preguntas para resolver con SQL:**

¿Cuál es la edad promedio de los clientes que han hecho un depósito?

¿Cuántos clientes tienen un préstamo personal (loan = 'yes') y también una hipoteca (housing = 'yes')?

¿Cuál es el balance promedio por nivel educativo?

¿Qué trabajos tienen la mayor tasa de conversión a depósitos?

¿Cuál es el mes con mayor número de contactos exitosos (deposit = 'yes')?

¿Qué porcentaje de los clientes tienen préstamos (housing o personales)?

¿Cuántas veces fueron contactadas las personas antes de aceptar (columna previous)?

¿Cuál es la duración promedio de las llamadas para los que aceptaron (deposit = 'yes')?

¿Qué combinación de estado civil y nivel educativo tiene mayor tasa de depósito?

¿Hay una relación entre la cantidad de campañas (campaign) y el resultado del depósito?

In [1]:
import pandas as pd
import sqlite3

df = pd.read_csv("bank.csv")

conn = sqlite3.connect(":memory:")
df.to_sql("bank", conn, index=False, if_exists="replace")

queries = [
    ("1. Edad promedio de clientes que hicieron un depósito",
     """
     SELECT AVG(age) AS avg_age
     FROM bank
     WHERE deposit = 'yes';
     """),

    ("2. Clientes con préstamo personal y con hipoteca",
     """
     SELECT COUNT(*) AS total
     FROM bank
     WHERE loan = 'yes' AND housing = 'yes';
     """),

    ("3. Balance promedio por nivel educativo",
     """
     SELECT education, ROUND(AVG(balance), 2) AS avg_balance
     FROM bank
     GROUP BY education
     ORDER BY avg_balance DESC;
     """),

    ("4. Trabajos con mayor tasa de conversión a depósito",
     """
     SELECT job,
            ROUND(100.0 * SUM(CASE WHEN deposit = 'yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS conversion_rate
     FROM bank
     GROUP BY job
     ORDER BY conversion_rate DESC;
     """),

    ("5. Mes con más contactos exitosos (deposit = 'yes')",
     """
     SELECT month, COUNT(*) AS successful_contacts
     FROM bank
     WHERE deposit = 'yes'
     GROUP BY month
     ORDER BY successful_contacts DESC;
     """),

    ("6. Porcentaje de clientes con préstamo (housing o personal)",
     """
     SELECT ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM bank), 2) AS loan_or_housing_pct
     FROM bank
     WHERE loan = 'yes' OR housing = 'yes';
     """),

    ("7. Contactos previos promedio antes del éxito",
     """
     SELECT AVG(previous) AS avg_previous_contacts
     FROM bank
     WHERE deposit = 'yes';
     """),

    ("8. Duración promedio de llamada para quienes aceptaron",
     """
     SELECT ROUND(AVG(duration), 2) AS avg_duration
     FROM bank
     WHERE deposit = 'yes';
     """),

    ("9. Estado civil y educación con mayor tasa de depósito",
     """
     SELECT marital, education,
            ROUND(100.0 * SUM(CASE WHEN deposit = 'yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS deposit_rate
     FROM bank
     GROUP BY marital, education
     ORDER BY deposit_rate DESC
     LIMIT 5;
     """),

    ("10. Relación entre cantidad de campañas y resultado del depósito",
     """
     SELECT campaign, COUNT(*) AS total_contacts,
            SUM(CASE WHEN deposit = 'yes' THEN 1 ELSE 0 END) AS successful_deposits
     FROM bank
     GROUP BY campaign
     ORDER BY campaign;
     """)
]

for title, sql in queries:
    print(f"\n--- {title} ---")
    df_result = pd.read_sql_query(sql, conn)
    display(df_result)



--- 1. Edad promedio de clientes que hicieron un depósito ---


,avg_age
0,41.67007



--- 2. Clientes con préstamo personal y con hipoteca ---


,total
0,835



--- 3. Balance promedio por nivel educativo ---


...

,avg_duration
0,537.29



--- 9. Estado civil y educación con mayor tasa de depósito ---


,marital,education,deposit_rate
0,single,tertiary,60.27
1,single,unknown,54.78
2,divorced,tertiary,52.59
3,divorced,primary,50.98
4,single,secondary,50.18



--- 10. Relación entre cantidad de campañas y resultado del depósito ---


,campaign,total_contacts,successful_deposits
0,1,4798,2561
1,2,3028,1401
2,3,1321,618
3,4,771,317
4,5,378,139
5,6,265,92
6,7,139,47
7,8,128,32
8,9,72,21
9,10,52,14
